# Advanced Models - Transfer Learning for Pneumonia Detection

This notebook trains multiple state-of-the-art models using transfer learning.

In [ ]:
import sys
sys.path.append('../src')

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Import our modules
from data_loader import MedicalImageLoader
from models import CNNModels

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

## 1. Data Preparation

In [ ]:
# Initialize data loader
data_dir = '../data'
img_size = (224, 224)
batch_size = 32

loader = MedicalImageLoader(data_dir, img_size=img_size, batch_size=batch_size)

# Create data generators
train_gen, val_gen, test_gen = loader.create_data_generators(validation_split=0.2)

print(f"Training samples: {train_gen.samples}")
print(f"Validation samples: {val_gen.samples}")
print(f"Test samples: {test_gen.samples}")

# Calculate class weights
class_weights = loader.get_class_weights(train_gen)
print(f"\nClass weights: {class_weights}")

## 2. Model Training - VGG16

In [ ]:
# Build VGG16 model
print("Building VGG16 model...")
vgg16_model = CNNModels.transfer_learning_model(
    base_model_name='VGG16',
    input_shape=(224, 224, 3),
    num_classes=1
)

vgg16_model.summary()

In [ ]:
# Train VGG16
epochs = 20
callbacks = CNNModels.get_callbacks('vgg16_transfer', patience=5)

history_vgg16 = vgg16_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs,
    class_weight=class_weights,
    callbacks=callbacks,
    verbose=1
)

# Save model
vgg16_model.save('../models/vgg16_final.h5')
print("VGG16 model saved!")

## 3. Model Training - ResNet50

In [ ]:
# Build ResNet50 model
print("Building ResNet50 model...")
resnet_model = CNNModels.transfer_learning_model(
    base_model_name='ResNet50',
    input_shape=(224, 224, 3),
    num_classes=1
)

# Train ResNet50
callbacks = CNNModels.get_callbacks('resnet50_transfer', patience=5)

history_resnet = resnet_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs,
    class_weight=class_weights,
    callbacks=callbacks,
    verbose=1
)

# Save model
resnet_model.save('../models/resnet50_final.h5')
print("ResNet50 model saved!")

## 4. Evaluate Models

In [ ]:
# Evaluate VGG16
print("Evaluating VGG16...")
vgg16_results = vgg16_model.evaluate(test_gen)
print(f"VGG16 Test Loss: {vgg16_results[0]:.4f}")
print(f"VGG16 Test Accuracy: {vgg16_results[1]:.4f}")
print(f"VGG16 Test Precision: {vgg16_results[2]:.4f}")
print(f"VGG16 Test Recall: {vgg16_results[3]:.4f}")

# Evaluate ResNet50
print("\nEvaluating ResNet50...")
resnet_results = resnet_model.evaluate(test_gen)
print(f"ResNet50 Test Loss: {resnet_results[0]:.4f}")
print(f"ResNet50 Test Accuracy: {resnet_results[1]:.4f}")
print(f"ResNet50 Test Precision: {resnet_results[2]:.4f}")
print(f"ResNet50 Test Recall: {resnet_results[3]:.4f}")

## 5. Generate Predictions

In [ ]:
# Get predictions
test_gen.reset()
y_true = test_gen.classes

y_pred_vgg16 = (vgg16_model.predict(test_gen) > 0.5).astype(int).flatten()
test_gen.reset()
y_pred_resnet = (resnet_model.predict(test_gen) > 0.5).astype(int).flatten()

print("Predictions generated!")

## 6. Confusion Matrices

In [ ]:
# Confusion matrices
cm_vgg16 = confusion_matrix(y_true, y_pred_vgg16)
cm_resnet = confusion_matrix(y_true, y_pred_resnet)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# VGG16
sns.heatmap(cm_vgg16, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['NORMAL', 'PNEUMONIA'],
            yticklabels=['NORMAL', 'PNEUMONIA'])
axes[0].set_title('VGG16 Confusion Matrix', fontsize=14, fontweight='bold')
axes[0].set_ylabel('True Label')
axes[0].set_xlabel('Predicted Label')

# ResNet50
sns.heatmap(cm_resnet, annot=True, fmt='d', cmap='Greens', ax=axes[1],
            xticklabels=['NORMAL', 'PNEUMONIA'],
            yticklabels=['NORMAL', 'PNEUMONIA'])
axes[1].set_title('ResNet50 Confusion Matrix', fontsize=14, fontweight='bold')
axes[1].set_ylabel('True Label')
axes[1].set_xlabel('Predicted Label')

plt.tight_layout()
plt.savefig('../results/confusion_matrices.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Training History

In [ ]:
# Plot training history
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# VGG16 Accuracy
axes[0, 0].plot(history_vgg16.history['accuracy'], label='Train')
axes[0, 0].plot(history_vgg16.history['val_accuracy'], label='Validation')
axes[0, 0].set_title('VGG16 Accuracy', fontweight='bold')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].legend()
axes[0, 0].grid(True)

# VGG16 Loss
axes[0, 1].plot(history_vgg16.history['loss'], label='Train')
axes[0, 1].plot(history_vgg16.history['val_loss'], label='Validation')
axes[0, 1].set_title('VGG16 Loss', fontweight='bold')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].legend()
axes[0, 1].grid(True)

# ResNet50 Accuracy
axes[1, 0].plot(history_resnet.history['accuracy'], label='Train')
axes[1, 0].plot(history_resnet.history['val_accuracy'], label='Validation')
axes[1, 0].set_title('ResNet50 Accuracy', fontweight='bold')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Accuracy')
axes[1, 0].legend()
axes[1, 0].grid(True)

# ResNet50 Loss
axes[1, 1].plot(history_resnet.history['loss'], label='Train')
axes[1, 1].plot(history_resnet.history['val_loss'], label='Validation')
axes[1, 1].set_title('ResNet50 Loss', fontweight='bold')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Loss')
axes[1, 1].legend()
axes[1, 1].grid(True)

plt.tight_layout()
plt.savefig('../results/training_history.png', dpi=300, bbox_inches='tight')
plt.show()

## 8. Results Summary

In [ ]:
# Create results table
results_df = pd.DataFrame({
    'Model': ['VGG16', 'ResNet50'],
    'Test Accuracy': [vgg16_results[1], resnet_results[1]],
    'Test Precision': [vgg16_results[2], resnet_results[2]],
    'Test Recall': [vgg16_results[3], resnet_results[3]],
    'Test Loss': [vgg16_results[0], resnet_results[0]]
})

# Calculate F1 scores
results_df['F1-Score'] = 2 * (results_df['Test Precision'] * results_df['Test Recall']) / \
                         (results_df['Test Precision'] + results_df['Test Recall'])

print("\nModel Performance Summary:")
print("=" * 80)
print(results_df.to_string(index=False))

# Save results
results_df.to_csv('../results/model_performance.csv', index=False)
print("\nResults saved to results/model_performance.csv")

## Summary

This notebook trained two state-of-the-art transfer learning models:
- **VGG16**: Deep CNN with 16 layers
- **ResNet50**: Residual network with skip connections

Both models were trained on 5,216 images with:
- Data augmentation
- Class weight balancing
- Early stopping
- Learning rate reduction

Results are saved in:
- `models/vgg16_final.h5`
- `models/resnet50_final.h5`
- `results/confusion_matrices.png`
- `results/training_history.png`
- `results/model_performance.csv`